# Working with SageMaker - Hands-On Industrial Internet of Things Veneri & Capasso - Packt 2018


This notebook has to be loaded on the SageMaker Notebook. We asume the image of the container already deployed in the ECS and dat on the bucket 'iiot-book-data'
Here the steps:
* working with data: we will remove unwanted engines
* saving data for training on input/data/training/
* training the model with train data
* deploy the endpoint
* test the endpoint


In [38]:
# Test bucket name immersionday-sagemaker-test
bucket_name = 'iiot-book-data'
file_name_train = 'train_FD001.txt'
file_name_test = 'test_FD001.txt'

## Working with Data

In [39]:
import boto3
import sagemaker

role = sagemaker.get_execution_role()
region = boto3.Session().region_name

print(region)
print(role)

import boto3

s3 = boto3.resource('s3')
s3.Bucket(bucket_name).download_file(file_name_train, file_name_train)
s3.Bucket(bucket_name).download_file(file_name_test,file_name_test)

eu-west-1
arn:aws:iam::266996497855:role/service-role/AmazonSageMaker-ExecutionRole-20180910T160262


In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# step 1: read the dataset
columns = ['unitid', 'time', 'set_1','set_2','set_3']
columns.extend(['sensor_' + str(i) for i in range(1,22)])
df = pd.read_csv(file_name_train, delim_whitespace=True,names=columns)
df_test = pd.read_csv(file_name_test, delim_whitespace=True,names=columns)

In [41]:
# only engine 1
i=1

# prepare model
columns_feature=['sensor_4','sensor_7']

file_name_train = 'train.csv'
file_name_test = 'test.csv'

dataset_train=df[(df.unitid ==i)]
dataset_test=df_test[(df_test.unitid ==i)]

### Saving training on S3
Now we can save only engine '1' data on S3 input/data/training/

In [42]:
import boto3
import os

s3 = boto3.resource('s3')
target_bucket = s3.Bucket(bucket_name)

np.savetxt(file_name_train, dataset_train, delimiter=",")
np.savetxt(file_name_test, dataset_test, delimiter=",")

with open(file_name_train, 'rb') as data:
    target_bucket.upload_fileobj(data, file_name_train)
    
with open(file_name_test, 'rb') as data:
    target_bucket.upload_fileobj(data, file_name_test)
    
with open(file_name_train, 'rb') as data:
    target_bucket.upload_fileobj(data, 'input/data/training/{}'.format(file_name_train))


## Traing Model on AWS SageMaker

In [43]:
import sagemaker as sage
role = sage.get_execution_role()
print(role)

arn:aws:iam::266996497855:role/service-role/AmazonSageMaker-ExecutionRole-20180910T160262


In [44]:
from time import gmtime, strftime

sess = sage.Session()

In [24]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
bucket_name = 'iiot-book-data3'

image = '{}.dkr.ecr.{}.amazonaws.com/rul-estimator:latest'.format(account, region)
output_location = 's3://{}/model'.format(bucket_name)
base_location = 's3://{}'.format(bucket_name)
print(image)
model = sage.estimator.Estimator(image,
                       role, 1, 'ml.m4.xlarge',
                       output_path=output_location,
                       sagemaker_session=sess)

model.fit(base_location)

INFO:sagemaker:Creating training-job with name: rul-estimator-2018-10-31-10-39-43-869


266996497855.dkr.ecr.eu-west-1.amazonaws.com/rul-estimator:latest
2018-10-31 10:39:44 Starting - Starting the training job...
2018-10-31 10:39:45 Starting - Launching requested ML instances......
2018-10-31 10:40:44 Starting - Preparing the instances for training...
2018-10-31 10:41:29 Downloading - Downloading input data
2018-10-31 10:41:29 Training - Downloading the training image...
2018-10-31 10:42:14 Training - Training image download completed. Training in progress..
Using TensorFlow backend.
Starting the training.
found ['/opt/ml/input/data/training/train.csv', '/opt/ml/input/data/training/test.csv']
[     unitid   time   set_1    ...      sensor_19  sensor_20  sensor_21
0       1.0    1.0 -0.0007    ...          100.0      39.06    23.4190
1       1.0    2.0  0.0019    ...          100.0      39.00    23.4236
2       1.0    3.0 -0.0043    ...          100.0      38.95    23.3442
3       1.0    4.0  0.0007    ...          100.0      38.88    23.3739
4       1.0    5.0 -0.0019   


2018-10-31 10:42:30 Uploading - Uploading generated training model
2018-10-31 10:42:30 Completed - Training job completed
Billable seconds: 68


In [45]:
print(model)

## Deploying the model

In [46]:

predictor = model.deploy(initial_instance_count=1,
                         instance_type='ml.m4.xlarge', 
                         endpoint_name='iiot-book-rul-predictor')

INFO:sagemaker:Creating model with name: rul-estimator-2018-10-31-10-56-47-427
INFO:sagemaker:Creating endpoint with name iiot-book-rul-predictor


---------------------------------------------------------------!

## Testing the model

In [33]:
import sagemaker
from sagemaker.predictor import csv_serializer, json_deserializer

#Use the following line to get a preditor by attaching to an existing endpoint
#linear_predictor = sagemaker.predictor.RealTimePredictor('<endpoint name>')

predictor.content_type = 'text/csv'
predictor.serializer = csv_serializer

result = predictor.predict(dataset_test.values).decode('utf-8')
print(result)


109.365486



## Clean the endpoint

In [34]:
predictor.delete_endpoint()

INFO:sagemaker:Deleting endpoint with name: iiot-book-rul-predictor
